## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-20-17-03-12 +0000,nyt,Iran Says U.S. Has Not Asked It to Stop Enrich...,https://www.nytimes.com/2026/02/20/world/middl...
1,2026-02-20-17-03-04 +0000,wapo,"Live updates: In rare rebuke of Trump, Supreme...",https://www.washingtonpost.com/national-securi...
2,2026-02-20-17-01-04 +0000,bbc,'Nasty' councillor jailed for coercive behaviour,https://www.bbc.com/news/articles/c627ngz1k1eo...
3,2026-02-20-17-00-26 +0000,bbc,Nasa targets early March to send humans back a...,https://www.bbc.com/news/articles/c86y1g6wde3o...
4,2026-02-20-17-00-00 +0000,wsj,How Streaming Became Cable TV’s Unlikely Life ...,https://www.wsj.com/business/media/how-streami...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2481/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,71
150,andrew,26
12,tariffs,25
632,epstein,22
278,prince,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
36,2026-02-20-16-09-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,196
331,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...,167
10,2026-02-20-16-54-32 +0000,nypost,Retail and apparel stocks spike after Supreme ...,https://nypost.com/2026/02/20/business/retail-...,163
109,2026-02-20-13-00-00 +0000,wsj,After President Trump’s first year back in off...,https://www.wsj.com/economy/trade/tariffs-comm...,162
1,2026-02-20-17-03-04 +0000,wapo,"Live updates: In rare rebuke of Trump, Supreme...",https://www.washingtonpost.com/national-securi...,154


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
36,196,2026-02-20-16-09-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...
177,131,2026-02-20-08-09-06 +0000,nyt,British Police Arrest Former Prince Andrew Ami...,https://www.nytimes.com/2026/02/19/world/europ...
109,66,2026-02-20-13-00-00 +0000,wsj,After President Trump’s first year back in off...,https://www.wsj.com/economy/trade/tariffs-comm...
62,61,2026-02-20-15-21-20 +0000,nypost,Hundreds of firms poised to be banned from sna...,https://nypost.com/2026/02/20/business/hundred...
192,58,2026-02-20-05-04-48 +0000,nypost,Les Wexner’s attorney caught on hot mic saying...,https://nypost.com/2026/02/20/us-news/les-wexn...
255,49,2026-02-20-00-00-00 +0000,wsj,The husband of embattled Labor Secretary Lori ...,https://www.wsj.com/politics/policy/labor-secr...
276,45,2026-02-19-22-59-54 +0000,nypost,UN nuclear watchdog chief urges US to secure I...,https://nypost.com/2026/02/19/world-news/un-nu...
30,38,2026-02-20-16-21-09 +0000,nypost,Husbands of Tahoe avalanche victims break sile...,https://nypost.com/2026/02/20/us-news/husbands...
208,37,2026-02-20-02-41-00 +0000,wsj,President Trump intends to order the release o...,https://www.wsj.com/politics/national-security...
96,37,2026-02-20-14-01-00 +0000,wsj,Key inflation metrics tracked by the Fed accel...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
